# North Carolina Scorecard

This notebook generates the county, school district, and statewide "future voter scorecards" for NC. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [6]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [7]:
# Inputs
as_of_data_date = pd.Timestamp("2025-08-23")
acs_year = '2022' # 2022 for 2024 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [8]:
# Derived variables
latest_18_yob = as_of_data_date.year - 18 # 2006 for 2024 scorecards
earliest_18_yob = latest_18_yob - 1 # 2005 for 2024 scorecards

latest_45_yob = as_of_data_date.year - 45 # 1979 for 2024 scorecards
earliest_45_yob = latest_45_yob - 1 # 1978 for 2024 scorecards

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

# Define columns

    # Voter file
REG_YOB_LATE_YEAR = 'REG_YOB_' + str(latest_18_yob) # col name for number of registrants with given birth year (2006 for 2024 scorecards)
REG_YOB_EARLY_YEAR = 'REG_YOB_' + str(earliest_18_yob) # col name for number of registrants with given birth year (2005 for 2024 scorecards)

REG_45_PLUS_YOB_LATE_YEAR = 'REG_45_PLUS_YOB_' + str(latest_45_yob) # col name for number of registrants with given birth year (1979 for 2024 scorecards)
REG_45_PLUS_YOB_EARLY_YEAR = 'REG_45_PLUS_YOB_' + str(earliest_45_yob) # col name for number of registrants with given birth year (1978 for 2024 scorecards)

    # ACS
EST_18_YO_THIS_YEAR = 'EST_18_YO_' + str(as_of_data_date.year) # col name for number of 18 yos this "as_of" year estimated from ACS
EST_18_AND_19_YO_THIS_YEAR = 'EST_18_AND_19_YO_' + str(as_of_data_date.year) # col name for number of 18 and 19 yos this "as_of" year estimated from ACS

EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [9]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
voter_source_table = data_date_suffix + "_nc_voter_registration_and_history"

acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants in a given birth year.
* Then, left joins the county estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [10]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_county AS(
    SELECT
    COUNTY_FIPS,
    COUNTY_NAME,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY COUNTY_FIPS, COUNTY_NAME

), acs_county AS(
    SELECT
    COUNTY_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_county.*,
acs_county.EST_15_TO_17_YO,
acs_county.MOE_15_TO_17_YO,
acs_county.""" + EST_18_YO_THIS_YEAR + """,
acs_county.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_county.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_county LEFT JOIN acs_county ON voter_file_county.COUNTY_FIPS = acs_county.COUNTY_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [11]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,101,JOHNSTON,143148,1358,2656,82573,80116,9984,98,3328.000000,6656.000000,103011
1,117,MARTIN,14201,114,219,9374,9201,753,32,251.000000,502.000000,13193
2,041,CHOWAN,8608,63,128,6083,5977,479,157,159.666667,319.333333,8604
3,121,MITCHELL,9896,63,116,6885,6784,534,52,178.000000,356.000000,9053
4,167,STANLY,40890,340,654,25129,24583,2391,35,797.000000,1594.000000,33220


In [12]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [13]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.3543956043956044


In [14]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [15]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250823,EST_REG_RATE_18_YO_AS_OF_20250823
29,183,WAKE,737650,8521,12765,400332,387492,47558,30,15852.666667,31705.333333,499266,13044.85989,0.822881
93,119,MECKLENBURG,682476,6991,10678,347221,336079,42911,18,14303.666667,28607.333333,465789,10775.236264,0.75332
63,081,GUILFORD,324748,3130,5306,190130,185264,21093,*****,7031.000000,14062.000000,253633,5010.423077,0.712619
11,067,FORSYTH,228204,2361,3649,135680,132450,15559,27,5186.333333,10372.666667,186737,3654.18956,0.704581
14,063,DURHAM,196625,1745,2555,100354,97388,10224,39,3408.000000,6816.000000,139455,2650.480769,0.777723


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [16]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.6438356164383562


In [17]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [18]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head(20)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250823,EST_REG_RATE_18_YO_AS_OF_20250823,EST_REG_45_PLUS_YO_AS_OF_20250823,EST_REG_RATE_45_PLUS_YO_AS_OF_20250823
29,183,WAKE,737650,8521,12765,400332,387492,47558,30,15852.666667,31705.333333,499266,13044.85989,0.822881,395758.849315,0.792681
93,119,MECKLENBURG,682476,6991,10678,347221,336079,42911,18,14303.666667,28607.333333,465789,10775.236264,0.75332,343252.616438,0.736927
63,081,GUILFORD,324748,3130,5306,190130,185264,21093,*****,7031.000000,14062.000000,253633,5010.423077,0.712619,188396.90411,0.742793
11,067,FORSYTH,228204,2361,3649,135680,132450,15559,27,5186.333333,10372.666667,186737,3654.18956,0.704581,134529.589041,0.720423
14,063,DURHAM,196625,1745,2555,100354,97388,10224,39,3408.000000,6816.000000,139455,2650.480769,0.777723,99297.616438,0.712041
88,021,BUNCOMBE,178728,1202,2111,109916,107023,8663,38,2887.666667,5775.333333,144295,1950.129121,0.67533,108885.616438,0.754604
5,051,CUMBERLAND,172260,1973,3032,96932,94267,12860,36,4286.666667,8573.333333,129853,3047.527473,0.710932,95982.821918,0.739165
75,179,UNION,156539,1915,3411,94121,91220,12970,5,4323.333333,8646.666667,115787,3123.843407,0.722554,93087.767123,0.803957
42,129,NEW HANOVER,155230,967,2140,93280,91056,7279,81,2426.333333,4852.666667,115498,1725.406593,0.711117,92487.890411,0.800775
0,101,JOHNSTON,143148,1358,2656,82573,80116,9984,98,3328.000000,6656.000000,103011,2299.274725,0.690888,81697.90411,0.793099


In [20]:
df_reg_est = df_reg_est.sort_values('EST_REG_RATE_18_YO_AS_OF_20250823', ascending=False)
df_reg_est

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250823,EST_REG_RATE_18_YO_AS_OF_20250823,EST_REG_45_PLUS_YO_AS_OF_20250823,EST_REG_RATE_45_PLUS_YO_AS_OF_20250823
68,177,TYRRELL,1869,6,21,1347,1318,36,39,12.000000,24.000000,1839,13.442308,1.120192,1336.671233,0.726847
12,189,WATAUGA,34021,225,424,17685,17259,1202,29,400.666667,801.333333,23535,375.263736,0.936598,17533.273973,0.744987
22,029,CAMDEN,7236,99,125,4444,4311,480,85,160.000000,320.000000,5604,143.299451,0.895622,4396.630137,0.784552
84,141,PENDER,44668,442,724,28222,27443,2379,130,793.000000,1586.000000,32899,698.582418,0.880936,27944.547945,0.849404
96,003,ALEXANDER,23087,266,341,15023,14732,1341,121,447.000000,894.000000,20246,386.848901,0.865434,14919.356164,0.736904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,137,PAMLICO,8942,43,86,6757,6673,436,78,145.333333,290.666667,8293,73.478022,0.505583,6727.082192,0.811176
40,173,SWAIN,8225,49,111,5464,5327,584,98,194.666667,389.333333,7158,88.337912,0.453791,5415.205479,0.756525
15,033,CASWELL,13433,59,157,9207,9046,783,46,261.000000,522.000000,13432,114.64011,0.439234,9149.657534,0.681184
74,103,JONES,6106,29,78,4173,4073,392,71,130.666667,261.333333,5813,56.642857,0.433491,4137.383562,0.711747


#### Output
Write back to BQ

##### Wide

In [21]:
# Flag largest counties
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.COUNTY_NAME.isin(df_reg_est.nlargest(10, columns='EST_18_YO_2025').COUNTY_NAME),1,0)

In [22]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


### Statewide Scorecard

In [23]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NC state, counting the number of registrants in a given birth year.
* Then, left joins the statewide estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos is derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [24]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_nc AS(
    SELECT
    STATE_FIPS,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY STATE_FIPS

), acs_nc AS(
    SELECT
    STATE_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_nc.*,
acs_nc.EST_15_TO_17_YO,
acs_nc.MOE_15_TO_17_YO,
acs_nc.""" + EST_18_YO_THIS_YEAR + """,
acs_nc.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_nc.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_nc LEFT JOIN acs_nc ON voter_file_nc.STATE_FIPS = acs_nc.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [25]:
# Preview
df

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,37,6495444,59771,103404,3894576,3798123,404849,794,134949.666667,269899.333333,5139416


In [26]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [27]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.3543956043956044


In [28]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [29]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250823,EST_REG_RATE_18_YO_AS_OF_20250823
0,37,6495444,59771,103404,3894576,3798123,404849,794,134949.666667,269899.333333,5139416,96416.923077,0.714466


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [30]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.6438356164383562


In [31]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [32]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250823,EST_REG_RATE_18_YO_AS_OF_20250823,EST_REG_45_PLUS_YO_AS_OF_20250823,EST_REG_RATE_45_PLUS_YO_AS_OF_20250823
0,37,6495444,59771,103404,3894576,3798123,404849,794,134949.666667,269899.333333,5139416,96416.923077,0.714466,3860222.876712,0.751101


#### Output
Write back to BQ

In [33]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
